In [37]:
import pandas as pd
import numpy as np 
import re
from bs4 import BeautifulSoup


#### Dataset from kaggle.com

In [25]:
path_kaggle = r"C:\Users\franc\ENTORNO TRABAJO DATA y BI\IRONHACK\Projecto Oscars\csv\raw\full_data.xlsx"
df_origin = pd.read_excel(path_kaggle)

##### Cleaning dataset from kaggle 

In [101]:
def cleaning_kaggle_info(df):
    """
    Cleans and transforms the Kaggle DataFrame.

    Args:
        df (pd.DataFrame): Original Kaggle DataFrame.

    Returns:
        pd.DataFrame: Cleaned and transformed DataFrame.
    """
    # Convert column names to lowercase
    df.columns = df.columns.str.lower()
    
    # Convert all in lowercase
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    
    # Drop unnecessary columns
    df.drop(columns=[ 
        'class', 
        'ceremony', 
        'nomid', 
        'name', 
        'nominees', 
        'nomineeids', 
        'detail', 
        'note', 
        'citation', 
        'multifilmnomination'
    ], inplace=True)
    
    # Filter out invalid years
    df = df[~df['year'].str.contains(r'/', regex=True)]
    df['year'] = df['year'].astype(int)
    
    # Filter the last 10 years
    df = df[df['year'] >= 2000]
    
    # Filter specific film categories
    film_categories = [
        'best picture',
        'animated feature film',
        'international feature film'
    ]
    df = df[df['canonicalcategory'].isin(film_categories)]
    
    # Convert 'winner' to 0 (no) and 1 (yes)
    df['winner'] = df['winner'].fillna(0).astype(int)
    
    # Convert 0 to 'no' and 1 to 'yes' in the 'winner' column
    df['winner'] = df['winner'].replace({0: 'no', 1: 'yes'})
    
    return df



In [102]:
# Call the function and save in csv directory

df_cleaned_kaggle = cleaning_kaggle_info(df_origin)
df_cleaned_kaggle.to_csv("csv/kaggle_clean.csv", index=False)


C:\Users\franc\AppData\Local\Temp\ipykernel_14316\1261603307.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


#### Obtain information from OMDB API 

In [29]:
import requests
import time
API_KEY = 'c251b987'
base_url = 'http://www.omdbapi.com/'

In [ ]:

def film_data(imdb_id):
    """
    Fetches movie data from the OMDB API for a given IMDb ID.

    Args:
        imdb_id (str): The IMDb ID of the movie to fetch data for.

    Returns:
        dict: A dictionary containing the following movie details:
            - filmid (str): The IMDb ID of the movie.
            - title (str): The title of the movie.
            - runtime (str): The runtime of the movie.
            - genre (str): The genre(s) of the movie.
            - director (str): The director(s) of the movie.
            - actors (str): The main actors in the movie.
            - language (str): The language(s) of the movie.
            - country (str): The country/countries of production.
            - imdbRating (float): The IMDb rating of the movie.
            - metascore (str): The Metascore of the movie.
            - imdbVotes (str): The number of votes on IMDb.

    If the API request fails, it prints an error message and returns None.
    """
    url = 'http://www.omdbapi.com/'
    params = {
        'apikey': API_KEY,
        'i': imdb_id
    }
    
    response = requests.get(url, params=params)
    
    print(f"Consulting information for IMDb ID: {imdb_id}")
    
    if response.status_code == 200:
        print(f"Successful response received for IMDb ID: {imdb_id}")
        data = response.json()
        
        imdb_rating = float(data.get('imdbRating')) if data.get('imdbRating') != 'N/A' else None
        title = data.get('Title')  # Movie title
        runtime = data.get('Runtime')  # Movie runtime
        genre = data.get('Genre')  # Genre(s)
        director = data.get('Director')  # Director(s)
        actors = data.get('Actors')  # Main actors
        language = data.get('Language')  # Language(s)
        country = data.get('Country')  # Country/countries of production
        metascore = data.get('Metascore')  # Metascore
        imdb_votes = data.get('imdbVotes')  # Number of IMDb votes

        return {
            'filmid': imdb_id,
            'title': title,
            'runtime': runtime,
            'genre': genre,
            'director': director,
            'actors': actors,
            'language': language,
            'country': country,
            'imdbRating': imdb_rating,
            'metascore': metascore,
            'imdbVotes': imdb_votes
        }
    else:
        print(f"Error fetching data for IMDb ID: {imdb_id}, status code: {response.status_code}")
        return None

In [31]:
# Test the function "film_data"
film_data('tt0245712')

Consultando información para el IMDb ID: tt0245712
Respuesta recibida con éxito para el Id de IMDb: tt0245712


{'filmid': 'tt0245712',
 'title': 'Amores Perros',
 'runtime': '154 min',
 'genre': 'Drama, Thriller',
 'director': 'Alejandro G. Iñárritu',
 'actors': 'Emilio Echevarría, Gael García Bernal, Goya Toledo',
 'language': 'Spanish',
 'country': 'Mexico',
 'imdbRating': 8.0,
 'metascore': '83',
 'imdbVotes': '261,019'}

In [ ]:
def films_df_imdb(df, archivo_salida='df_imdb.csv'):
    """
    Fetches movie data from the OMDb API for a list of IMDb IDs, creates a DataFrame, and saves it to a CSV file.

    Args:
        df (pd.DataFrame): DataFrame containing a column 'filmid' with IMDb IDs.
        archivo_salida (str): Path to save the resulting DataFrame as a CSV file.

    Returns:
        pd.DataFrame: DataFrame containing movie details fetched from the OMDb API.
    """
    datos = []
    ids = df['filmid'].unique()
    total = len(ids)

    print(f"📡 Starting query for {total} unique titles from OMDb...")

    for i, imdb_id in enumerate(ids, start=1):
        print(f"🔍 ({i}/{total}) Querying IMDb ID: {imdb_id}")
        datos.append(film_data(imdb_id))
        time.sleep(0.25)  # Pause to avoid API rate limits

    print("✅ Query completed. Generating DataFrame...")
    df_info = pd.DataFrame(datos)
    print(f"💾 Saving results to file: {archivo_salida}")
    df_info.to_csv(archivo_salida, index=False)
    return df_info

In [33]:
df_imdb = films_df_imdb(df_cleaned_kaggle)
df_imdb.to_csv("csv/raw/imdb_data.csv", index=False)

📡 Iniciando consulta de 407 títulos únicos a OMDb...
🔍 (1/407) Consultando IMDb ID: tt0245712
Consultando información para el IMDb ID: tt0245712
Respuesta recibida con éxito para el Id de IMDb: tt0245712
🔍 (2/407) Consultando IMDb ID: tt0190332
Consultando información para el IMDb ID: tt0190332
Respuesta recibida con éxito para el Id de IMDb: tt0190332
🔍 (3/407) Consultando IMDb ID: tt0234288
Consultando información para el IMDb ID: tt0234288
Respuesta recibida con éxito para el Id de IMDb: tt0234288
🔍 (4/407) Consultando IMDb ID: tt0209037
Consultando información para el IMDb ID: tt0209037
Respuesta recibida con éxito para el Id de IMDb: tt0209037
🔍 (5/407) Consultando IMDb ID: tt0216787
Consultando información para el IMDb ID: tt0216787
Respuesta recibida con éxito para el Id de IMDb: tt0216787
🔍 (6/407) Consultando IMDb ID: tt0241303
Consultando información para el IMDb ID: tt0241303
Respuesta recibida con éxito para el Id de IMDb: tt0241303
🔍 (7/407) Consultando IMDb ID: tt0195685


### Web Scraping de BoxOfficeMojo


#### Worlwide Boxoffice

In [ ]:
# Función para obtener recaudación internacional desde Box Office Mojo
def film_world_boxoffice(imdb_id):
    """
    Fetches the worldwide box office revenue for a given IMDb ID from Box Office Mojo.

    Args:
        imdb_id (str): The IMDb ID of the movie to fetch data for.

    Returns:
        str: The worldwide box office revenue as a string (e.g., "$20,000,000").
             Returns None if the revenue is not found or if an error occurs.
    """
    url = f'https://www.boxofficemojo.com/title/{imdb_id}/?ref_=bo_se_r_1'
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error accessing the page for IMDb ID: {imdb_id} - {e}")
        return 'Error accessing the page'

    soup = BeautifulSoup(response.text, 'html.parser')
    print(f"✅ Successfully received response for IMDb ID: {imdb_id}")

    try:
        # Search all performance summary sections
        performance_sections = soup.select('div.mojo-performance-summary-table div.a-section.a-spacing-none')
        for section in performance_sections:
            if 'Worldwide' in section.get_text():
                money_span = section.find('span', class_='money')
                if money_span:
                    return money_span.get_text(strip=True)
        return None
    except Exception as e:
        print(f"❌ Error processing the HTML: {e}")
        return None


In [38]:
#Test function "film_world_boxoffice"
film_world_boxoffice('tt0848228')

✅ Respuesta recibida con éxito para el Id de IMDb: tt0848228


'$1,520,538,536'

In [ ]:
# Función para extraer IMDb ID, título y recaudación internacional
def worldwide_boxoffice_df(df):
    """
    Extracts worldwide box office revenue for a list of IMDb IDs, creates a DataFrame, and returns it.

    Args:
        df (pd.DataFrame): DataFrame containing a column 'filmid' with IMDb IDs and a column 'film' with movie titles.

    Returns:
        pd.DataFrame: A DataFrame containing the following columns:
            - 'IMDb ID': The IMDb ID of the movie.
            - 'title': The title of the movie.
            - 'Worlwide boxoffice': The worldwide box office revenue as a string (e.g., "$20,000,000").
    """
    datos = []
    ids = df['filmid'].unique()
    total = len(ids)

    for i, imdb_id in enumerate(ids, start=1):
        print(f"🌍 ({i}/{total}) Querying worldwide box office for IMDb ID: {imdb_id}")
        recaudacion = film_world_boxoffice(imdb_id)
        time.sleep(0.25)
        
        # Get the corresponding title from the original DataFrame
        titulo = df[df['filmid'] == imdb_id]['film'].values[0]

        # Save the data in the list
        datos.append({
            'IMDb ID': imdb_id,
            'title': titulo,
            'Worlwide boxoffice': recaudacion
        })

    # Create a DataFrame with the results
    df_worldwide = pd.DataFrame(datos)
    return df_worldwide

In [42]:
# Ejecutar la función para obtener los datos y generar un archivo CSV
df_worldwide = worldwide_boxoffice_df(df_cleaned_kaggle)

# Guardar los resultados en un archivo CSV separado
df_worldwide.to_csv("csv/raw/worlwide_boxoffice.csv", index=False)

print('✅ Proceso completado. Datos guardados en worlwide_boxoffice.csv.')

🌍 (1/407) Consultando recaudación para IMDb ID: tt0245712
✅ Respuesta recibida con éxito para el Id de IMDb: tt0245712
🌍 (2/407) Consultando recaudación para IMDb ID: tt0190332
✅ Respuesta recibida con éxito para el Id de IMDb: tt0190332
🌍 (3/407) Consultando recaudación para IMDb ID: tt0234288
✅ Respuesta recibida con éxito para el Id de IMDb: tt0234288
🌍 (4/407) Consultando recaudación para IMDb ID: tt0209037
✅ Respuesta recibida con éxito para el Id de IMDb: tt0209037
🌍 (5/407) Consultando recaudación para IMDb ID: tt0216787
✅ Respuesta recibida con éxito para el Id de IMDb: tt0216787
🌍 (6/407) Consultando recaudación para IMDb ID: tt0241303
✅ Respuesta recibida con éxito para el Id de IMDb: tt0241303
🌍 (7/407) Consultando recaudación para IMDb ID: tt0195685
✅ Respuesta recibida con éxito para el Id de IMDb: tt0195685
🌍 (8/407) Consultando recaudación para IMDb ID: tt0172495
✅ Respuesta recibida con éxito para el Id de IMDb: tt0172495
🌍 (9/407) Consultando recaudación para IMDb ID: 

#### Domestic Boxoffice

In [ ]:
# Función para obtener recaudación domestic desde Box Office Mojo
def film_domestic_boxoffice(imdb_id):
    """
    Fetches the domestic box office revenue for a given IMDb ID from Box Office Mojo.

    Args:
        imdb_id (str): The IMDb ID of the movie to fetch data for.

    Returns:
        str: The domestic box office revenue as a string (e.g., "$20,000,000").
             Returns None if the revenue is not found or if an error occurs.
    """
    url = f'https://www.boxofficemojo.com/title/{imdb_id}/?ref_=bo_se_r_1'
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error accessing the page for IMDb ID: {imdb_id} - {e}")
        return 'Error accessing the page'

    soup = BeautifulSoup(response.text, 'html.parser')
    print(f"✅ Successfully received response for IMDb ID: {imdb_id}")

    try:
        # Search all performance summary sections
        performance_sections = soup.select('div.mojo-performance-summary-table div.a-section.a-spacing-none')
        for section in performance_sections:
            if 'Domestic' in section.get_text():
                money_span = section.find('span', class_='money')
                if money_span:
                    return money_span.get_text(strip=True)
        return None
    except Exception as e:
        print(f"❌ Error processing the HTML: {e}")
        return None

In [44]:
#Test Funcion  
film_domestic_boxoffice('tt0848228')

✅ Respuesta recibida con éxito para el Id de IMDb: tt0848228


'$623,357,910'

In [ ]:
# Función para extraer IMDb ID, título y recaudación domestic
def films_domestic_boxoffice_df(df):
    """
    Extracts domestic box office revenue for a list of IMDb IDs, creates a DataFrame, and returns it.

    Args:
        df (pd.DataFrame): DataFrame containing a column 'filmid' with IMDb IDs and a column 'film' with movie titles.

    Returns:
        pd.DataFrame: A DataFrame containing the following columns:
            - 'IMDb ID': The IMDb ID of the movie.
            - 'title': The title of the movie.
            - 'domestic boxoffice': The domestic box office revenue as a string (e.g., "$20,000,000").
    """
    datos = []
    ids = df['filmid'].unique()
    total = len(ids)

    for i, imdb_id in enumerate(ids, start=1):
        print(f"🌍 ({i}/{total}) Querying domestic box office for IMDb ID: {imdb_id}")
        recaudacion = film_domestic_boxoffice(imdb_id)
        time.sleep(0.25)
        
        # Get the corresponding title from the original DataFrame
        titulo = df[df['filmid'] == imdb_id]['film'].values[0]

        # Save the data in the list
        datos.append({
            'IMDb ID': imdb_id,
            'title': titulo,
            'domestic boxoffice': recaudacion
        })

    # Create a DataFrame with the results
    df_domestic_boxoffice = pd.DataFrame(datos)
    return df_domestic_boxoffice

In [46]:
# Ejecutar la función para obtener los datos y generar un archivo CSV
df_domestic_boxoffice = films_domestic_boxoffice_df(df_cleaned_kaggle)

# Guardar los resultados en un archivo CSV separado
df_domestic_boxoffice.to_csv('csv/raw/domestic_boxoffice.csv', index=False)

print('✅ Proceso completado. Datos guardados en domestic_boxoffice.csv.')

🌍 (1/407) Consultando recaudación para IMDb ID: tt0245712
✅ Respuesta recibida con éxito para el Id de IMDb: tt0245712
🌍 (2/407) Consultando recaudación para IMDb ID: tt0190332
✅ Respuesta recibida con éxito para el Id de IMDb: tt0190332
🌍 (3/407) Consultando recaudación para IMDb ID: tt0234288
✅ Respuesta recibida con éxito para el Id de IMDb: tt0234288
🌍 (4/407) Consultando recaudación para IMDb ID: tt0209037
✅ Respuesta recibida con éxito para el Id de IMDb: tt0209037
🌍 (5/407) Consultando recaudación para IMDb ID: tt0216787
✅ Respuesta recibida con éxito para el Id de IMDb: tt0216787
🌍 (6/407) Consultando recaudación para IMDb ID: tt0241303
✅ Respuesta recibida con éxito para el Id de IMDb: tt0241303
🌍 (7/407) Consultando recaudación para IMDb ID: tt0195685
✅ Respuesta recibida con éxito para el Id de IMDb: tt0195685
🌍 (8/407) Consultando recaudación para IMDb ID: tt0172495
✅ Respuesta recibida con éxito para el Id de IMDb: tt0172495
🌍 (9/407) Consultando recaudación para IMDb ID: 

#### International Boxoffice

In [ ]:

# Function to obtain international box office revenue from Box Office Mojo
def film_internacional_boxoffice(imdb_id):
    """
    Fetches the international box office revenue for a given IMDb ID from Box Office Mojo.

    Args:
        imdb_id (str): The IMDb ID of the movie to fetch data for.

    Returns:
        str: The international box office revenue as a string (e.g., "$20,000,000").
            Returns None if the revenue is not found or if an error occurs.
    """
    url = f'https://www.boxofficemojo.com/title/{imdb_id}/?ref_=bo_se_r_1'
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        performance_sections = soup.select('div.mojo-performance-summary-table div.a-section.a-spacing-none')
        for section in performance_sections:
            if 'International' in section.get_text():
                money_span = section.find('span', class_='money')
                if money_span:
                    return money_span.get_text(strip=True)
        return None
    except Exception as e:
        print(f"❌ Error procesando el HTML: {e}")
        return None

In [48]:
#test function "film_internacional_boxoffice"
film_internacional_boxoffice('tt0848228')

✅ Respuesta recibida con éxito para el Id de IMDb: tt0848228


'$897,180,626'

In [ ]:
# Función para extraer IMDb ID, título y recaudación international
def films_international_boxoffice_df(df):
    """
    Extracts international box office revenue for a list of IMDb IDs, creates a DataFrame, and returns it.

    Args:
        df (pd.DataFrame): DataFrame containing a column 'filmid' with IMDb IDs and a column 'film' with movie titles.

    Returns:
        pd.DataFrame: A DataFrame containing the following columns:
            - 'IMDb ID': The IMDb ID of the movie.
            - 'title': The title of the movie.
            - 'international boxoffice': The international box office revenue as a string (e.g., "$20,000,000").
    """
    datos = []
    ids = df['filmid'].unique()
    total = len(ids)

    for i, imdb_id in enumerate(ids, start=1):
        print(f"🌍 ({i}/{total}) Querying international box office for IMDb ID: {imdb_id}")
        recaudacion = film_internacional_boxoffice(imdb_id)
        time.sleep(0.25)
        
        # Get the corresponding title from the original DataFrame
        titulo = df[df['filmid'] == imdb_id]['film'].values[0]

        # Save the data in the list
        datos.append({
            'IMDb ID': imdb_id,
            'title': titulo,
            'international boxoffice': recaudacion
        })

    # Create a DataFrame with the results
    df_international_boxoffice = pd.DataFrame(datos)
    return df_international_boxoffice

In [56]:
# Ejecutar la función para obtener los datos y generar un archivo CSV
df_international_boxoffice = films_international_boxoffice_df(df_cleaned_kaggle)

# Guardar los resultados en un archivo CSV separado
df_international_boxoffice.to_csv('csv/raw/international_boxoffice.csv', index=False)

print('✅ Proceso completado. Datos guardados en domestic_boxoffice.csv.')

🌍 (1/407) Consultando recaudación para IMDb ID: tt0245712
✅ Respuesta recibida con éxito para el Id de IMDb: tt0245712
🌍 (2/407) Consultando recaudación para IMDb ID: tt0190332
✅ Respuesta recibida con éxito para el Id de IMDb: tt0190332
🌍 (3/407) Consultando recaudación para IMDb ID: tt0234288
✅ Respuesta recibida con éxito para el Id de IMDb: tt0234288
🌍 (4/407) Consultando recaudación para IMDb ID: tt0209037
✅ Respuesta recibida con éxito para el Id de IMDb: tt0209037
🌍 (5/407) Consultando recaudación para IMDb ID: tt0216787
✅ Respuesta recibida con éxito para el Id de IMDb: tt0216787
🌍 (6/407) Consultando recaudación para IMDb ID: tt0241303
✅ Respuesta recibida con éxito para el Id de IMDb: tt0241303
🌍 (7/407) Consultando recaudación para IMDb ID: tt0195685
✅ Respuesta recibida con éxito para el Id de IMDb: tt0195685
🌍 (8/407) Consultando recaudación para IMDb ID: tt0172495
✅ Respuesta recibida con éxito para el Id de IMDb: tt0172495
🌍 (9/407) Consultando recaudación para IMDb ID: 

##### Create boxoffice dataset

In [ ]:

def create_boxoffice_dataset(df_domestic_boxoffice, df_international_boxoffice, df_worldwide):
    """
        Creates a combined DataFrame of box office revenues from three sources:
        domestic, international, and worldwide.

        Args:
            df_domestic_boxoffice (pd.DataFrame): Domestic box office revenue data.
            df_international_boxoffice (pd.DataFrame): International box office revenue data.
            df_worldwide (pd.DataFrame): Worldwide box office revenue data.

        Returns:
            pd.DataFrame: Combined and cleaned DataFrame.
        """
    # Eliminar columna 'title' si existe
    for df in [df_international_boxoffice, df_worldwide]:
        if 'title' in df.columns:
            df.drop(columns='title', inplace=True)

    # Establecer 'IMDb ID' como índice
    df_domestic_boxoffice.set_index('IMDb ID', inplace=True)
    df_international_boxoffice.set_index('IMDb ID', inplace=True)
    df_worldwide.set_index('IMDb ID', inplace=True)

    # Unir los tres DataFrames
    df_boxoffice = df_domestic_boxoffice.join(
        [df_international_boxoffice, df_worldwide], how='outer'
    ).reset_index()

    # Limpiar y convertir columnas monetarias
    columnas_recaudacion = [
        'domestic boxoffice', 'international boxoffice', 'Worlwide boxoffice'
    ]
    
    for col in columnas_recaudacion:
        df_boxoffice[col] = (
            df_boxoffice[col]
            .replace(r'[\$,]', '', regex=True)
            .replace('', np.nan)
            .fillna(0)
            .astype(int)
        )

    # Renombrar columna 'IMDb ID' a 'filmid'
    df_boxoffice.rename(columns={'IMDb ID': 'filmid'}, inplace=True)

    return df_boxoffice


In [ ]:
df_boxoffice = create_boxoffice_dataset(
    df_domestic_boxoffice, df_international_boxoffice, df_worldwide
)

df_boxoffice.to_csv('csv/raw/boxoffice_data.csv', index=False)

### Web scraping www.the-numbers.com
-create the badget dataset

In [62]:
def film_url_fixed(film_name):
    """
    Converts a film name into a URL-friendly format by applying the following transformations:
    - Removes leading articles ("The ", "A ") and appends them at the end (e.g., "The Matrix" -> "Matrix, The").
    - Replaces special characters such as "&" with "and".
    - Replaces spaces, colons, commas, question marks, slashes, and parentheses with hyphens ("-").
    - Removes periods, apostrophes, and exclamation marks.

    Args:
        film_name (str): The original name of the film.

    Returns:
        str: The transformed, URL-friendly film name.
    """
    film_name = film_name.strip()

    if film_name.startswith("The "):
        film_name = film_name[4:] + ", The"
    elif film_name.startswith("A "):
        film_name = film_name[2:] + ", A"

    film_name = film_name.replace("&", "and")
    for char in [" ", ":", ",",  "?","/", "(", ")"]:
        film_name = film_name.replace(char, "-")
    for char1 in [".", "'", "!",]:
        film_name = film_name.replace(char1, "")
    return film_name

In [ ]:
def film_budget(film_name):
    """
    Fetches the production budget of a film from The Numbers website.

    Args:
        film_name (str): The name of the film.

    Returns:
        int: The production budget of the film in dollars, or None if not found or an error occurs.
    """
    film_url = film_url_fixed(film_name)
    url = f'https://www.the-numbers.com/movie/{film_url}#tab=summary'

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        b_tags = soup.find_all('b', string='Production Budget:')
        for b in b_tags:
            td_parent = b.find_parent('td')
            if td_parent:
                td_siguiente = td_parent.find_next_sibling('td')
                if td_siguiente:
                    texto_completo = td_siguiente.text.strip()
                    texto_dinero = texto_completo.split(' (')[0]  # "$2,000,000"
                    # Clean and convert to int
                    presupuesto = int(texto_dinero.replace('$', '').replace(',', ''))
                    print(f"   ✅ Budget found: {presupuesto}")
                    return presupuesto
        print("   ⚠️ Budget not found")
        return None
    else:
        print("   ❌ Error accessing the page")
        return None

In [64]:
# Ejemplo de uso
print(film_budget("Amores Perros"))

   ✅ Presupuesto encontrado: 2000000
2000000


In [ ]:
def films_budget_df(df):
    """
    Creates a DataFrame containing the budget information for a list of films.

    Args:
        df (pd.DataFrame): DataFrame containing a column 'filmid' with IMDb IDs and a column 'title' with movie titles.

    Returns:
        pd.DataFrame: A DataFrame containing the following columns:
            - 'IMDb ID': The IMDb ID of the movie.
            - 'title': The title of the movie.
            - 'budget': The production budget of the movie.
    """
    datos = []
    ids = df['filmid'].unique()
    total = len(ids)

    for i, imdb_id in enumerate(ids, start=1):
        print(f"🎬 ({i}/{total}) Consulting budget for IMDb ID: {imdb_id}")
        
        # Get the corresponding title from the original DataFrame
        titulo = df[df['filmid'] == imdb_id]['title'].values[0]

        # Get the budget
        presupuesto = film_budget(titulo)
        time.sleep(0.25)

        # Save the data in the list
        datos.append({
            'IMDb ID': imdb_id,
            'title': titulo,
            'budget': presupuesto
        })

    # Create a DataFrame with the results
    df_budget = pd.DataFrame(datos)
    return df_budget

In [67]:
df_budget = films_budget_df(df_imdb)

# Guardar los resultados en un archivo CSV
df_budget.to_csv('csv/raw/movie_budgets.csv', index=False)
print("✅ Datos guardados en 'movie_budgets.csv'")

🎬 (1/407) Consultando presupuesto para IMDb ID: tt0245712
   ✅ Presupuesto encontrado: 2000000
🎬 (2/407) Consultando presupuesto para IMDb ID: tt0190332
   ✅ Presupuesto encontrado: 15000000
🎬 (3/407) Consultando presupuesto para IMDb ID: tt0234288
   ⚠️ Presupuesto no encontrado
🎬 (4/407) Consultando presupuesto para IMDb ID: tt0209037
   ⚠️ Presupuesto no encontrado
🎬 (5/407) Consultando presupuesto para IMDb ID: tt0216787
   ⚠️ Presupuesto no encontrado
🎬 (6/407) Consultando presupuesto para IMDb ID: tt0241303
   ✅ Presupuesto encontrado: 25000000
🎬 (7/407) Consultando presupuesto para IMDb ID: tt0195685
   ✅ Presupuesto encontrado: 50000000
🎬 (8/407) Consultando presupuesto para IMDb ID: tt0172495
   ✅ Presupuesto encontrado: 103000000
🎬 (9/407) Consultando presupuesto para IMDb ID: tt0181865
   ✅ Presupuesto encontrado: 48000000
🎬 (10/407) Consultando presupuesto para IMDb ID: tt0268397
   ✅ Presupuesto encontrado: 25000000
🎬 (11/407) Consultando presupuesto para IMDb ID: tt019878

In [68]:
presupuestos = {
    'Amélie': 10000000,
    'Lagaan: Once Upon a Time in India': 9000000,
    'Spirited Away': 19200000,
    'The Crime of Padre Amaro': 3000000,
    'The Triplets of Belleville': 10000000,
    'The Barbarian Invasions': 4500000,
    'The Chorus': 13000000,
    'The Sea Inside': 11500000,
    'Yesterday': 26000000,
    'The Aviator': 190000000,
    'Howl\'s Moving Castle': 19000000,
    'Sophie Scholl: The Final Days': 6000000,
    'After the Wedding': 7000000,
    'Days of Glory': 20000000,
    'The Lives of Others': 2000000,
    'Pan\'s Labyrinth': 19000000,
    'The Departed': 90000000,
    'The Counterfeiters': 5000000,
    'Mongol: The Rise of Genghis Khan': 20000000,
    'WALL·E': 180000000,
    'The Baader Meinhof Complex': 23000000,
    'The Class': 7000000,
    'Departures': 4500000,
    'The Milk of Sorrow': 2000000,
    'A Prophet': 10000000,
    'The Secret in Their Eyes': 4000000,
    'The White Ribbon': 11000000,
    'The Blind Side': 29000000,
    'An Education': 7000000,
    'Precious': 10000000,
    'The Illusionist': 16000000,
    'In a Better World': 6000000,
    'The Fighter': 25000000,
    'The Social Network': 40000000,
    'A Cat in Paris': 6000000,
    'Bullhead': 3000000,
    'Footnote': 1500000,
    'A Separation': 3000000,
    'The Tree of Life': 32000000,
    'A Royal Affair': 10000000,
    'War Witch': 2000000,
    'Les Misérables': 61000000,
    'Ernest & Celestine': 8000000,
    'The Wind Rises': 35000000,
    'The Great Beauty': 13000000,
    'The Hunt': 4000000,
    'The Missing Picture': 1000000,
    'The Tale of The Princess Kaguya': 49000000,
    'Tangerines': 1500000,
    'Wild Tales': 3000000,
    'Birdman or (The Unexpected Virtue of Ignorance)': 18000000,
    'The Theory of Everything': 15000000,
    'The Boy and the World': 2000000,
    'Shaun the Sheep Movie': 25000000,
    'When Marnie Was There': 20000000,
    'Embrace of the Serpent': 1200000,
    'Son of Saul': 1500000,
    'A War': 5000000,
    'The Martian': 108000000,
    'The Revenant': 135000000,
    'My Life as a Zucchini': 8000000,
    'The Red Turtle': 6000000,
    'Land of Mine': 6000000,
    'A Man Called Ove': 7000000,
    'The Salesman': 2000000,
    'A Fantastic Woman': 2000000,
    'The Insult': 4000000,
    'Loveless': 5000000,
    'On Body and Soul': 3000000,
    'The Square': 10000000,
    'Darkest Hour': 30000000,
    'Lady Bird': 10000000,
    'Mirai': 9000000,
    'Spider-Man: Into the Spider-Verse': 90000000,
    'Capernaum': 4000000,
    'Never Look Away': 30000000,
    'Roma': 15000000,
    'Shoplifters': 3000000,
    'Bohemian Rhapsody': 52000000,
    'The Favourite': 15000000,
    'Green Book': 23000000,
    'A Star Is Born': 36000000,
    'How to Train Your Dragon: The Hidden World': 129000000,
    'I Lost My Body': 6000000,
    'Klaus': 40000000,
    'Missing Link': 100000000,
    'Honeyland': 500,
    'Pain and Glory': 11000000,
    'Parasite': 11400000,
    'The Irishman': 160000000,
    'Jojo Rabbit': 14000000,
    '1917': 95000000,
    'Onward': 200000000,
    'Over the Moon': 38000000,
    'A Shaun the Sheep Movie: Farmageddon': 25000000,
    'Soul': 150000000,
    'Wolfwalkers': 10000000,
    'Another Round': 4000000,
    'Better Days': 9000000,
    'Collective': 1000000,
    'The Man Who Sold His Skin': 1500000,
    'Quo Vadis, Aida?': 2300000,
    'The Father': 6000000,
    'Encanto': 120000000,
    'Flee': 3000000,
    'Luca': 50000000,
    'The Mitchells vs. the Machines': 85000000,
    'Drive My Car': 2000000,
    'The Hand of God': 5000000,
    'Lunana: A Yak in the Classroom': 200,
    'The Worst Person in the World': 5000000,
    'Belfast': 20000000,
    'CODA': 10000000,
    'Dune: Part One': 165000000,
    'King Richard': 50000000,
    'Licorice Pizza': 40000000,
    'The Power of the Dog': 39000000,
    'Guillermo del Toro\'s Pinocchio': 35000000,
    'Marcel the Shell with Shoes On': 6300000,
    'Puss in Boots: The Last Wish': 90000000,
    'The Sea Beast': 70000000,
    'Turning Red': 175000000,
    'Argentina, 1985': 10000000,
    'Close': 3000000,
    'EO': 2000000,
    'The Quiet Girl': 1000000,
    'Avatar: The Way of Water': 460000000,
    'The Banshees of Inisherin': 20000000,
    'Elvis': 85000000,
    'The Fabelmans': 40000000,
    'Tár': 35000000,
    'Top Gun: Maverick': 170000000,
    'Women Talking': 10000000,
    'The Boy and the Heron': 25000000,
    'Elemental': 175000000,
    'Robot Dreams': 50000000,
    'Spider-Man: Across the Spider-Verse': 100000000,
    'Io Capitano': 3000000,
    'Perfect Days': 4000000,
    'Society of the Snow': 10000000,
    'The Teachers\' Lounge': 4000000,
    'The Zone of Interest': 8000000,
    'American Fiction': 7000000,
    'Anatomy of a Fall': 8000000,
    'The Holdovers': 25000000,
    'Oppenheimer': 100000000,
    'Past Lives': 2000000,
    'Poor Things': 50000000,
    'Flow': 5000000,
    'Inside Out 2': 200000000,
    'Memoir of a Snail': 1000000,
    'Wallace & Gromit: Vengeance Most Fowl': 30000000,
    'The Wild Robot': 40000000,
    'Emilia Pérez': 3000000,
    'The Girl with the Needle': 2000000,
    'The Seed of the Sacred Fig': 1500000,
    'Anora': 2000000,
    'The Brutalist': 12000000,
    'A Complete Unknown': 2000000,
    'Conclave': 10000000,
    'Dune: Part Two': 200000000,
    'Nickel Boys': 4000000,
    'The Substance': 6000000
}

##### Data cleaning budgets

In [106]:

def clean_budget(df_budget, presupuestos, ruta_salida='movie_budgets_clean.csv'):
    """
        Replaces budget values in the DataFrame based on the `presupuestos` dictionary,
        fills missing values with 0, renames 'IMDb ID' to 'filmid', and saves the result to a CSV.

        Args:
            df_budget (pd.DataFrame): DataFrame with columns 'title' and 'budget'.
            presupuestos (dict): Dictionary with budgets by title.
            ruta_salida (str): Path to the output CSV file.
        """
    df_budget=pd.read_csv('csv/raw/movie_budgets.csv')
    # Reemplazar presupuestos donde haya valores en el diccionario
    df_budget['budget'] = df_budget['title'].map(presupuestos).fillna(df_budget['budget'])
    df_budget['budget'] = df_budget['budget'].fillna(0)
    df_budget['title'] = df_budget['title'].str.lower()

    # Renombrar columna IMDb ID si existe
    if 'IMDb ID' in df_budget.columns:
        df_budget.rename(columns={'IMDb ID': 'filmid'}, inplace=True)

    # Guardar el archivo actualizado
    df_budget.to_csv(ruta_salida, index=False)
    print(f"✅ Archivo guardado como: {ruta_salida}")


In [107]:
df_budget = clean_budget(df_budget, presupuestos, 'csv/movie_budgets_clean.csv')


✅ Archivo guardado como: csv/movie_budgets_clean.csv


#### Create financial dataset

In [ ]:
def create_financial_data(df_budget, df_boxoffice, ruta_salida='financial_data.csv'):
    """
    Creates a financial dataset containing the revenue and budget of the movies,
    calculates the ROI, and saves the result to a CSV file.

    Args:
        df_budget (pd.DataFrame): DataFrame with information about the movie budgets.
        df_boxoffice (pd.DataFrame): DataFrame with the revenue of the movies.
        ruta_salida (str): Path to the output CSV file.
    """
    df_budget=pd.read_csv('csv/movie_budgets_clean.csv')
    # Eliminar la columna 'title' si existe
    for df in [df_budget]:
        if 'title' in df.columns:
            df.drop(columns='title', inplace=True)

    # Establecer 'IMDb ID' como índice
    df_budget.set_index('filmid', inplace=True)
    df_boxoffice.set_index('filmid', inplace=True)

    # Unir los DataFrames de presupuesto y recaudación por 'IMDb ID'
    df_financial_info = df_boxoffice.join(
        df_budget,
        how='inner'
    ).reset_index()

    # Calcular el ROI
    df_financial_info['ROI'] = (
        (df_financial_info['Worlwide boxoffice'] - df_financial_info['budget']) / df_financial_info['budget']
    ).round(2)

    # Guardar el archivo actualizado
    df_financial_info.to_csv(ruta_salida, index=False)
    print(f"✅ Archivo guardado como: {ruta_salida}")


In [ ]:
create_financial_data(df_budget, df_boxoffice, 'csv/financial_data.csv')


In [ ]:
def create_final_dataset(df_imdb, df_financial_data, df_kaggle, ruta_salida='final_dataset.csv'):
    """
    Creates the final dataset by merging IMDb, revenue, and awards data, 
    cleaning unnecessary columns, and saving the resulting CSV file.

    Args:
        df_imdb (pd.DataFrame): DataFrame with IMDb information.
        df_financial_data (pd.DataFrame): DataFrame with financial information (revenue, budget, ROI).
        df_kaggle (pd.DataFrame): DataFrame with awards and category information.
        ruta_salida (str): Path to the output CSV file.
    """
    # Eliminar columnas si existen en df_financial_data
    columnas_a_eliminar = {'title', 'domestic boxoffice', 'international boxoffice'}
    if columnas_a_eliminar.issubset(df_financial_data.columns):
        df_financial_data.drop(columns=columnas_a_eliminar, inplace=True)

    # Eliminar la columna 'filmid' de df_financial_data si existe (para evitar duplicados)
    if 'filmid' in df_financial_data.columns:
        df_financial_data.drop(columns='filmid', inplace=True)

    # Hacer inner join entre IMDb y la información financiera
    df_final = df_imdb.join(df_financial_data, how='inner')

    # Unir con el DataFrame de Kaggle (filmid como índice)
    df_final = df_final.join(df_kaggle.set_index('filmid')[['year', 'winner', 'category']], on='filmid', how='inner')

    df_final['genre'] = df_final['genre'].str.split(',').str[0].str.strip()
    df_final['country'] = df_final['country'].str.split(',').str[0].str.strip()
    # Guardar el archivo final
    df_final.to_csv(ruta_salida, index=False)
    print(f"✅ Archivo final guardado como: {ruta_salida}")



In [105]:
df_financial_data =pd.read_csv('csv/financial_data.csv')
create_final_dataset(df_imdb, df_financial_data, df_cleaned_kaggle, 'csv/final_dataset.csv')


✅ Archivo final guardado como: csv/final_dataset.csv
